Výpočet tepové frekvence

In [ ]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt

#algoritmus pro detekci R vrcholů v signálech + prezentace tepové frekvence při jednotlivých měřeních
def Nacti_soubor(cesta_k_souboru): return wfdb.rdrecord(cesta_k_souboru)

# nejdřív si načtu jednotlivá měření, abych věděl, jak to vypadá
ecg_mereni = Nacti_soubor(r"C:\Users\Patrik\OneDrive\Plocha\PZS\seminarni_prace_1\mereni_signalu\100001\100001_ECG")
#print("Informace o EKG měření:", ecg_mereni.__dict__) 

#vezmu si ze souboru jednotlivé signály
ecg_signal = ecg_mereni.p_signal
#print(ecg_signal[::50]) #testování

plt.figure(figsize=(10, 4))
plt.plot(ecg_signal)
plt.title("EKG Signál")
plt.xlabel("Čas (s)")
plt.ylim([0,10])
plt.ylabel("Amplituda [uV]")
plt.show() #!se dlouho načítá!

#určím si parametry pro výpočet
vzorkovaci_f = 5000 #vzorkovací frekvence, jak často budu zaznamenávat
amplitudy = ecg_signal[:, 0]; 
dolni_mez = 500
x_coords = [i for i in range(len(amplitudy)) if amplitudy[i] > dolni_mez]
prumer_x = sum(x_coords) / len(x_coords)
print(f"průměr: {prumer_x}")
min_peaks_height = 10000 #zvolím si minimální výšku vrcholů, která je ideální dané ekg
min_distance = int(vzorkovaci_f) #minimální vzdálenost mezi vrcholi bude vzorkovací frekvence

#nyní si vytvořím algoritmus na detekci r vrcholů
r_peaks = [] #seznam, kam si uložím souřadnice vrcholů
for i in range(1, len(ecg_signal)-1):
    if ecg_signal[i]>min_peaks_height:
        #zkontroluji že hodnota je větší něž okolní hodnoty
        if ecg_signal[i]>ecg_signal[i-1] and ecg_signal[i]>ecg_signal[i+1]:
            if len(r_peaks) == 0 or (i-r_peaks[-1]>=min_distance): 
                r_peaks.append(i)
print(r_peaks[:20]) #testování

#r_peaks_hodnoty = [ecg_signal[i] if i in r_peaks else None for i in range(len(ecg_signal))]


print(len(ecg_signal))
time_period = 8

tepova_f = ((len(r_peaks)*60) / time_period) #výpočet tepové frekvence v BPM
#zaokrouhlena_tepova_f = round(tepova_f,2) #zaokrouhlím na celé číslo
print(time_period,tepova_f)
plt.figure(figsize=(20,4))
plt.plot(r_peaks,[ecg_signal[i] for i in r_peaks], linestyle='None', marker='o', color='green', label="R-vrcholy")
plt.xlabel("Čas (s)")
plt.ylabel("Amplituda [uV]")
plt.title(f"Tepová frekvence {tepova_f:.2f} bpm")
plt.ylim([0,25000]) #nastavím si x od 0 do 25000
plt.show()